In [1]:
from scipy.optimize import minimize

loading data from the sql 

In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

 # Load environment variables from .env file
load_dotenv()

# Retrieve the database URL from the environment variable
database_url = os.getenv("DATABASE_URL")

# --- PostgreSQL connection ---
# Create the engine using the environment variable
engine = create_engine(database_url)

# --- Load historical price data ---
query = "SELECT date, ticker, close FROM financial_data"
df = pd.read_sql(query, engine)
df['date'] = pd.to_datetime(df['date'])

# --- Pivot to get price matrix [date x ticker] ---
price_df = df.pivot(index='date', columns='ticker', values='close').sort_index()
price_df = price_df.dropna(axis=1)  # Drop tickers with missing data

# --- Compute log returns ---
log_returns = np.log(price_df / price_df.shift(1)).dropna()

# ✅ Preview
print("Log returns shape:", log_returns.shape)
print(log_returns.head())


Log returns shape: (1342, 92)
ticker          AAPL      ADBE       ADI       ADP       AEP      ALGN  \
date                                                                     
2020-01-03 -0.009769 -0.007865 -0.017760 -0.002116 -0.001071 -0.011487   
2020-01-06  0.007936  0.005710 -0.011818  0.001352  0.003315  0.019212   
2020-01-07 -0.004714 -0.000959  0.022496 -0.012178  0.000214 -0.009913   
2020-01-08  0.015958  0.013348  0.008991  0.009353 -0.002993  0.010333   
2020-01-09  0.021019  0.007607  0.000000  0.008800  0.005019  0.036190   

ticker          AMAT       AMD      AMGN      AMZN  ...      TEAM      TSLA  \
date                                                ...                       
2020-01-03 -0.016044 -0.010236 -0.006812 -0.012213  ...  0.001301  0.029203   
2020-01-06 -0.021801 -0.004330  0.007645  0.014776  ...  0.037096  0.019072   
2020-01-07  0.028477 -0.002897 -0.009449  0.002089  ...  0.003596  0.038067   
2020-01-08 -0.000649 -0.008743  0.000756 -0.007839  ... 

calculating expected returns & covariance matirx 

In [3]:
# Assuming 252 trading days per year
expected_returns = log_returns.mean() * 252

# --- Compute annualized covariance matrix ---
cov_matrix = log_returns.cov() * 252

# --- Preview ---
print("✅ Expected Returns (annualized):")
print(expected_returns.head())

print("\n✅ Covariance Matrix (annualized):")
print(cov_matrix.iloc[:5, :5])  # Show a 5x5 slice


✅ Expected Returns (annualized):
ticker
AAPL    0.188583
ADBE    0.025454
ADI     0.109049
ADP     0.129259
AEP     0.061917
dtype: float64

✅ Covariance Matrix (annualized):
ticker      AAPL      ADBE       ADI       ADP       AEP
ticker                                                  
AAPL    0.107145  0.078212  0.074367  0.049137  0.026762
ADBE    0.078212  0.153232  0.081179  0.052999  0.020500
ADI     0.074367  0.081179  0.136338  0.055804  0.022214
ADP     0.049137  0.052999  0.055804  0.076025  0.033876
AEP     0.026762  0.020500  0.022214  0.033876  0.060987


In [4]:
# --- Prepare data ---
tickers = expected_returns.index.tolist()
mu = expected_returns.values
cov = cov_matrix.values
n_assets = len(tickers)

# --- Objective function: portfolio variance ---
def portfolio_variance(weights, cov_matrix):
    return weights.T @ cov_matrix @ weights

# --- Constraints: fully invested ---
constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]

# --- Bounds: long-only portfolio (0 <= w <= 1) ---
bounds = [(0.0, 1.0) for _ in range(n_assets)]

# --- Initial guess: equal weighting ---
w0 = np.ones(n_assets) / n_assets

# --- Solve optimization ---
result = minimize(portfolio_variance, w0, args=(cov,), method='SLSQP',
                  bounds=bounds, constraints=constraints)

# --- Extract optimal weights ---
optimal_weights = pd.Series(result.x, index=tickers)

# --- Display result ---
print("✅ Optimal Weights (Minimum Variance Portfolio):")
print(optimal_weights[optimal_weights > 0].sort_values(ascending=False))


✅ Optimal Weights (Minimum Variance Portfolio):
KDP      1.792759e-01
GILD     1.400223e-01
AEP      1.286571e-01
AZN      1.027372e-01
COST     1.010826e-01
EA       9.360079e-02
PCAR     5.675488e-02
REGN     3.058176e-02
MAR      3.016989e-02
NTES     2.292809e-02
TTWO     2.291702e-02
MDLZ     2.273615e-02
KHC      2.021744e-02
TCOM     1.759889e-02
ORLY     1.574066e-02
DLTR     9.148201e-03
PANW     3.991574e-03
AMGN     1.839588e-03
ALGN     7.616022e-17
ADBE     4.373931e-17
AVGO     4.004152e-17
ADP      2.379537e-17
AMD      2.289658e-17
AXON     2.260266e-17
CRWD     1.915568e-17
DDOG     1.605241e-17
ADI      1.401951e-17
ROST     9.522408e-18
BIIB     9.174376e-18
MSTR     8.630234e-18
CSX      8.420393e-18
DXCM     7.926573e-18
SMCI     7.896942e-18
EXC      7.894988e-18
SNPS     7.506409e-18
MELI     6.806854e-18
GOOGL    5.791159e-18
INTU     5.033897e-18
CDW      4.838094e-18
HON      4.444113e-18
CPRT     4.436563e-18
XEL      3.646240e-18
INTC     3.309203e-18
MSFT  


- These are the optimal asset weights (allocations) that minimize the total portfolio variance, assuming:
    - Full investment (sum = 1)
    - No short selling (weights ≥ 0)
- Assets with very small values (e.g., 1e-17) are essentially excluded from the portfolio — their weight is effectively zero.

Calculating for Constrained Portfolio Optimization (Target Return + Max Weight)

In [5]:
# --- Target return: e.g., at least 5% annual return ---
target_return = 0.05

# --- Objective: minimize portfolio variance ---
def portfolio_variance(weights, cov_matrix):
    return weights.T @ cov_matrix @ weights

# --- Constraints: fully invested + target return ---
constraints = [
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # weights sum to 1
    {'type': 'ineq', 'fun': lambda w: w @ mu - target_return}  # return constraint
]

# --- Bounds: long-only + max 10% in any asset ---
max_weight = 0.10
bounds = [(0.0, max_weight) for _ in range(n_assets)]

# --- Initial guess ---
w0 = np.ones(n_assets) / n_assets

# --- Solve ---
result_constrained = minimize(portfolio_variance, w0, args=(cov,), method='SLSQP',
                              bounds=bounds, constraints=constraints)

# --- Extract optimal weights ---
optimal_weights_constrained = pd.Series(result_constrained.x, index=tickers)

# --- Display non-zero weights ---
print("✅ Constrained Optimal Weights:")
print(optimal_weights_constrained[optimal_weights_constrained > 0].sort_values(ascending=False))

✅ Constrained Optimal Weights:
GILD     1.000000e-01
COST     1.000000e-01
AEP      1.000000e-01
EA       1.000000e-01
KDP      1.000000e-01
AZN      1.000000e-01
PCAR     6.624339e-02
MDLZ     5.976794e-02
REGN     4.738087e-02
KHC      4.419196e-02
MAR      3.335460e-02
ORLY     3.006222e-02
NTES     2.788272e-02
TTWO     2.481639e-02
AMGN     2.246592e-02
TCOM     1.831156e-02
DLTR     1.390943e-02
XEL      6.550361e-03
PANW     3.646570e-03
WBA      1.416080e-03
CTSH     2.764456e-17
CMCSA    2.720223e-17
ZS       2.208065e-17
FTNT     2.096983e-17
QCOM     1.846692e-17
AAPL     1.803440e-17
NXPI     1.707646e-17
AVGO     1.610220e-17
DOCU     1.514091e-17
CSCO     1.468939e-17
EXC      1.466164e-17
DDOG     1.460557e-17
CDNS     1.459777e-17
CTAS     1.227257e-17
TEAM     1.179056e-17
DXCM     1.118828e-17
TSLA     1.038379e-17
OKTA     9.335249e-18
GOOGL    8.941136e-18
ROST     8.246247e-18
AMD      7.842767e-18
MNST     7.378066e-18
SNPS     6.149388e-18
KLAC     5.876706e-18
C


- Enforced param:
    - Target return ≥ 5%
    - Max weight = 10% per asset
    - Long-only constraint (no negative weights)
- As a result:
    - Assets like EA, AZN, KDP, AEP, and GILD hit the maximum allowed allocation of 10% — they are likely low-risk, high-return contributors.
    - Several other assets have near-zero weights (e.g., MRVL, MSTR) — the optimizer deems them suboptimal under your constraints.


Saving connstrained weights to the sql 


In [6]:
from sqlalchemy import create_engine

# --- Prepare final weights table ---
weights_df = optimal_weights_constrained.reset_index()
weights_df.columns = ['ticker', 'weight']
weights_df = weights_df[weights_df['weight'] > 0].sort_values(by='weight', ascending=False)

# --- Save to PostgreSQL ---
# Load environment variables from .env file
load_dotenv()

# Retrieve the database URL from the environment variable
database_url = os.getenv("DATABASE_URL")

# --- PostgreSQL connection ---
# Create the engine using the environment variable
engine = create_engine(database_url)
weights_df.to_sql("optimal_portfolio_weights", engine, if_exists='replace', index=False)

print("✅ Optimal weights saved to PostgreSQL table: optimal_portfolio_weights")

✅ Optimal weights saved to PostgreSQL table: optimal_portfolio_weights
